In [1]:
import pymongo

In [2]:
import pandas as pd
import numpy as np

In [3]:
client = pymongo.MongoClient(
    host=['<>',
          '<>',
          '<>'],
    replicaSet='triggr-shard-0',
    ssl=True,
    authSource='admin',
    username='<>',
    password='<>'
)

In [5]:
db = client['extractData']

In [6]:
collection = db['categories_10']

In [6]:
collection.count()

11

In [7]:
mapping = pd.read_excel('job_title_association.xlsx')

In [8]:
mapping.head()

,Founder,C-Suite,Management,Director,Sales,Marketing,Development / IT,Finance,Strategy,Support,Human Resources,Legal,Power,No Power
0,Founder,Chief,Manager,Director,BDM,Growth,Computer Engineer,Fraud,Product,Coordinator,People,Lawyer,Senior,Assistant
1,Co-Founder,CEO,Head,VP,Business Development,Communications,Software,Accountant,Analyst,Customer,Recruiter,Legal,Chief,Junior
2,CoFounder,COO,GM,Vice President,Sales,Public Relations,Information Technology,Actuary,Project,Office,Recruitment,Law,Head,Co-ordinator
3,Co Founder,CIO,General Manager,President,Relationship,Social,Technical,CFO,Producer,Admin,Human Resources,Counsel,Boss,Coordinator
4,NaN,CTO,Mgmt,NaN,Account Management,SEO,Web,Risk,Strategic,NaN,Training,Arbitrator,Director,Admin


In [9]:
from fuzzywuzzy import fuzz

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
'ICT Networks & Help Desk Partner'.split()

In [9]:
sent_tokenize('Organisational Excellence - Information Services')

NameError: name 'sent_tokenize' is not defined

In [10]:
def categorize(title):
    
    groups = []
    if not title:
        return groups
    
    def _match(column, maching_title):
        matching_list = mapping[column].dropna().str.lower()
        for keyword in matching_list:
            keywords = keyword.strip().split()
            if len(keywords) == 1:
                for word in title.lower().split():
                    if keyword == word:
                        #print keyword, word
                        return True
            elif len(keywords) > 1 and keyword in title.lower():
                #print keyword, title
                return True
        return False
    
    for col in mapping.columns[:-2]:
        if _match(col, title):
            groups.append(col)
    
    if _match('No Power', title):
        groups.append('No Power')
    elif _match('Power', title):
        groups.append('Power')
    return groups

In [11]:
categorize('ASSISTANT educator')

['No Power']

In [12]:
categorize('COO')

[u'C-Suite', 'Power']

In [13]:
categorize('Organisational Excellence - Information Services')

[]

In [14]:
categorize('Acquisitions & Special Projects Manager')

[u'Management']

### Scan collection

In [13]:
collection.count()

164453

In [34]:
collection_all = db['categories.fin']

In [15]:
# create test sample
for doc in db['categories.fin'].find().limit(10):
    db['categories_10'].replace_one({'_id': doc[ '_id']}, doc)

In [ ]:
# delete title_group
for doc in collection_all.find({'employees': {'$exists': True, '$ne': []}}):
    collection.update_one({'_id': doc['_id']}, {'$set': {'employees.title_group': []}})

In [51]:
from collections import OrderedDict

In [79]:
from bson.son import SON

In [26]:
for d in db['categorise_10'].find({"employees": {"$elemMatch": {"id": 188400010}}}):
    print d

In [ ]:
doc.up

In [ ]:
from pymongo import UpdateOne

In [ ]:
i = 0
table = 'categories.new'
for doc in db[table].find({'employees': {'$exists': True, '$ne': []}, 
                                          'employees.title': {'$ne': ''}, 
                                          'employees.title_group': {'$exists': False}},
                          {'_id': 1, 'employees': 1}):
    if i % 100 == 0:
        print 'Finished {}'.format(i)
    
    for employee in doc.get('employees'):
        employee['title_group'] = categorize(employee.get('title'))
        
    db[table].update_one({'_id': doc['_id']}, {'$set': {'employees': doc['employees']}})
        
    i += 1

Finished 0
Finished 100
Finished 200
Finished 300
Finished 400
Finished 500
Finished 600
Finished 700
Finished 800
Finished 900


In [55]:
doc['_id']

ObjectId('5a532b4b7d726f3a6a394596')

In [76]:
collection.update_one({'_id': doc[ '_id']}, {'$push': {'e': []}})

In [ ]:
import nltk

In [ ]:
nltk.download()

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    score /= count
    return score
 
sentences = [
    "Dogs are awesome.",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "Cats are beautiful animals.",
]
 
focus_sentence = "Cats are beautiful animals."
 
for sentence in sentences:
    print "Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence))
    print "Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence))
    print 

In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
%time model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  


In [ ]:
pprint.pprint(model.most_similar(positive=['woman', 'king'], negative=['man']))

In [ ]:
model.

In [ ]:
model.cosine_similarities('woman', 'king')

In [ ]:
import logging
import pprint

# for logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Load evaluation dataset of analogy task 
model.accuracy('questions-words.txt')
# execute analogy task like king - man + woman = queen
pprint.pprint(model.most_similar(positive=['woman', 'king'], negative=['man']))

In [85]:
for doc in collection.find().limit(10):
    print doc['_id']
    
    employees = []
    for employee in doc.get('employees', []):
        print employee.get('title', '')
        print employee.get('title_group', '')

5a619fc03d4d9662e0c17662
5a532b4b7d726f3a6a394596
Customer and service Delivery Manager Barwon Western Districts
[u'Management', u'Support']
Assistant educator
[u'No Power']
Front Desk Receptionist
[]
Disability Specialist
[]
Administrative Assistant
[u'No Power']
occupational therapist
[]
Community Educator
[]
Occupational Therapist
[]
Contract Services Manager
[u'Management', u'Legal']
Disability Support Worker
[u'No Power']
Community Capacity Building Project Officer
[u'Strategy']
Disability Support Worker
[u'No Power']
Occupational Therapist
[]
occupational therapist
[]
Acting coordinator
[u'Support', u'No Power']
Speech Pathologist
[]
Research Assistant
[u'No Power']
Business Planning Assistant
[u'No Power']
Communication Access Consultant
[]
DSW; Trainer
[]
Coordinator
[u'Support', u'No Power']
Dsw
[]
Cleaner
[]
Coordinator
[u'Support', u'No Power']
Senior Speech Pathologist
[u'Power']
Admin Coordinator
[u'Support', u'No Power']
Human Rights Project Manager
[u'Management', u'Stra